<a href="https://colab.research.google.com/github/mithun0808/python/blob/main/Rule_Based_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
import nltk
import re
from nltk.chat.util import Chat, reflections

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
pairs = [
    [r"my name is (.*)", ["Hello %1, how can I assist you today?"]],
    [r"hi|hey|hello", ["Hello, how can I help you?", "Hey there! What can I do for you?",
                        "Hi! How can I assist you today?"]],
    [r"what is your name?", ["I am a chatbot created to assist you. You can call me Chatbot."]],
    [r"how are you?", ["I'm a bot, so I don't have feelings, but I'm here to help you!"]],
    [r"can you help me with (.*)", ["Sure, I can help you with %1. Please provide more details."]],
    [r"sorry (.*)", ["It's okay. How can I assist you?"]],
    [r"thank you|thanks", ["You're welcome!", "No problem!", "Happy to help!"]],
    [r"quit", ["Bye! Have a great day!", "Goodbye!"]],
    [r"(.*)", ["I'm sorry, I don't understand that. Can you rephrase?",
               "Could you please elaborate on that?"]]
]

In [15]:
class RBChatbot:
  def __init__(self, pairs):
    self.chatbot = Chat(pairs, reflections)

  def respond(self, user_input):
    return self.chatbot.respond(user_input)


chatbot = RBChatbot(pairs)

In [16]:
def chat_with_bot():
  print("Hi I'm your chatbot Type 'quit' to exit")
  while True:
    user_input = input("You:")
    if user_input.lower() == "quit":
      print("Chatbot: Bye! Have a great day!")
      break
    response = chatbot.respond(user_input)
    print(f"Chatbot:{response}")

chat_with_bot()

Hi I'm your chatbot Type 'quit' to exit
You:hello
Chatbot:Hello, how can I help you?
You:hi
Chatbot:Hey there! What can I do for you?
You:what is your name
Chatbot:I am a chatbot created to assist you. You can call me Chatbot.
You:how will u assist me
Chatbot:Could you please elaborate on that?
You:what you want
Chatbot:I'm sorry, I don't understand that. Can you rephrase?
You:quit
Chatbot: Bye! Have a great day!


In [19]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [21]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

model_name = "deepset/bert-base-cased-squad2"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
context = """The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower."""
question = "Who designed the Eiffel Tower?"

inputs = tokenizer(question, context, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

In [25]:
start_idx = torch.argmax(start_scores)
end_idx = torch.argmax(end_scores) + 1

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx]))

print(f"Answer: {answer}")

Answer: Gustave Eiffel


In [26]:
context1 = """Coronavirus disease 2019 (COVID-19, also known as SARS-2) is a contagious disease
caused by the coronavirus SARS-CoV-2. In January 2020, the disease spread worldwide,
resulting in the COVID-19 pandemic."""
question1 = "What is the cause for coronavirus?"

inputs = tokenizer(question1, context1, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

In [28]:
start_idx = torch.argmax(start_scores)
end_idx = torch.argmax(end_scores) + 1

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx]))

print(f"Answer: {answer}")

Answer: SARS - CoV - 2
